# Cut Whole Slide Images into SpatialData objects

In [1]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
from pathlib import Path

import pandas as pd
from loguru import logger

from plex_pipe.core_cutting.channel_scanner import discover_channels
from plex_pipe.core_cutting.controller import CorePreparationController
from plex_pipe.core_cutting.file_io import LocalFileStrategy
from plex_pipe.utils.config_loaders import load_analysis_settings

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env2\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env2\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


## Read in config

In [2]:
# load analysis configuration
config_path = r'../examples/example_pipeline_config.yaml'

config = load_analysis_settings(config_path)

2026-02-06 14:50:51.837 | WARNING  | plex_pipe.processors.base:_log_missing_optional_params:51 - InstansegSegmenter.Params: Parameter overlap not provided. Using default value: 80.


## Define the logger

In [3]:
log_file = config.log_dir_path / f"cores_cutting_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

## Define ROIs for processing

In [4]:
df_path = config.roi_info_file_path

df = pd.read_pickle(df_path)
df.head()

,roi_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,ROI_000,256.0,5056.0,256.0,5120.0,rectangle,"[[256.0, 5120.0], [5056.0, 5120.0], [5056.0, 2..."
1,ROI_001,128.0,4992.0,6912.0,11776.0,rectangle,"[[128.0, 11776.0], [4992.0, 11776.0], [4992.0,..."


In [5]:
# # create a subset of cores (optional)
# df = df[:1]
# df

## Discover signal channels

In [6]:
channel_map = discover_channels(Path(config.general.image_dir),
                                include_channels=config.roi_cutting.include_channels,
                                exclude_channels=config.roi_cutting.exclude_channels,
                                use_markers=config.roi_cutting.use_markers,
                                ignore_markers=config.roi_cutting.ignore_markers)

2026-02-06 14:51:00.632 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:76 - Discovered 4 channels:
2026-02-06 14:51:00.633 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:78 - 001_CD45 <- ..\examples\input\sample_1.0.4_R000_Cy5_CD45-AF647_FINAL_AFR_F.tiff
2026-02-06 14:51:00.633 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:78 - 001_DAPI <- ..\examples\input\sample_1.0.4_R000_DAPI__FINAL_F.tiff
2026-02-06 14:51:00.633 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:78 - 001_NaKATPase <- ..\examples\input\sample_1.0.4_R000_Cy7_NaKATPase-AF750_FINAL_AFR_F.tiff
2026-02-06 14:51:00.633 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:78 - 001_bCat <- ..\examples\input\sample_1.0.4_R000_Cy3_bCat-AF555_FINAL_AFR_F.tiff
2026-02-06 14:51:00.633 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:128 - Ignoring markers = ['bCat']
2

In [7]:
# # for tests you can request a smaller set of channels here
# # then send 'short_map' to the controller instead of 'channel_map'
# selected_keys = ["DAPI"]
# short_map = {k: channel_map[k] for k in selected_keys if k in channel_map}
# short_map

## Run ROI cutting

In [10]:
strategy = LocalFileStrategy()

controller = CorePreparationController(
    metadata_df = df, # df defines which cores to process
    image_paths = channel_map, # defines which channels to use
    temp_dir = config.roi_dir_tif_path,
    output_dir = config.roi_dir_output_path,
    file_strategy = strategy,
    margin = config.roi_cutting.margin,
    mask_value = config.roi_cutting.mask_value,
    max_pyramid_levels = config.sdata_storage.max_pyramid_level,
    chunk_size = config.sdata_storage.chunk_size,
    downscale = config.sdata_storage.chunk_size,
    core_cleanup_enabled = False,
    )


controller.run()

2026-02-06 14:52:37.618 | INFO     | plex_pipe.core_cutting.controller:run:115 - Starting controller run loop...
2026-02-06 14:52:37.618 | INFO     | plex_pipe.core_cutting.controller:run:125 - Channel CD45 file available at ..\examples\input\sample_1.0.4_R000_Cy5_CD45-AF647_FINAL_AFR_F.tiff.
2026-02-06 14:52:37.700 | DEBUG    | plex_pipe.core_cutting.controller:cut_channel:16 - Cut and saved ROI ROI_000, channel CD45.
2026-02-06 14:52:37.777 | DEBUG    | plex_pipe.core_cutting.controller:cut_channel:16 - Cut and saved ROI ROI_001, channel CD45.
2026-02-06 14:52:37.777 | DEBUG    | plex_pipe.core_cutting.controller:cut_channel:20 - Closed file handle for channel CD45.
2026-02-06 14:52:37.778 | INFO     | plex_pipe.core_cutting.controller:run:125 - Channel DAPI file available at ..\examples\input\sample_1.0.4_R000_DAPI__FINAL_F.tiff.
2026-02-06 14:52:37.874 | DEBUG    | plex_pipe.core_cutting.controller:cut_channel:16 - Cut and saved ROI ROI_000, channel DAPI.
2026-02-06 14:52:37.964 | 